In [1]:
## reading the data
import pandas as pd
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [2]:
## process and drop irrelevant columns
df=df.drop(['RowNumber','CustomerId', 'Surname'],axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.Geography.unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [5]:
## encode categorical columns
from sklearn.preprocessing import LabelEncoder, StandardScaler
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df.Gender.unique()

array([0, 1])

In [32]:
## one hot encoding for Geography column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder(sparse_output=False).set_output(transform='pandas')
geo_encoder=onehot_encoder_geo.fit_transform(df[['Geography']])

In [33]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [34]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [35]:
geo_encoder

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [37]:
df=pd.concat([df.drop('Geography', axis=1), geo_encoder], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [38]:
## save the encoder and scaler
import pickle

with open("label_encoder_gender.pkl", "wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo, file)    

In [39]:
## divide data into independent and dependent data

X=df.drop("Exited", axis=1) ## independent data
y=df["Exited"]

## train test split 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.2, random_state=42)


## scaled the data

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [40]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [41]:
X_test

array([[-0.57749609,  0.91324755, -0.6557859 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.29729735,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.52560743, -1.09499335,  0.48508334, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.81311987, -1.09499335,  0.77030065, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.41876609,  0.91324755, -0.94100321, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.24540869,  0.91324755,  0.00972116, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [42]:
## save scaler transformation into pickle file

with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler,file)

In [46]:
X_train.shape

(8000, 12)

## ANN implementation

In [47]:
import tensorflow 
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard # type: ignore
import datetime


In [49]:
## Building ANN model
model=Sequential([
                Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## first hidden layer
                Dense(32, activation='relu'),   ## second hidden layer
                Dense(1,activation='sigmoid')   ## Output layer
])

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [53]:
## complile the model
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [54]:
## Setup Tensorboard
log_dir="logs/fit/" + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [55]:
## Earlystoping
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [56]:
## Train the model
history=model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=100, callbacks=[early_stopping_callback, tensorflow_callback])

Epoch 1/100


250/250 [==============================] - 9s 13ms/step - loss: 0.3882 - accuracy: 0.8385 - val_loss: 0.3600 - val_accuracy: 0.8525
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3576 - accuracy: 0.8526 - val_loss: 0.3424 - val_accuracy: 0.8610
Epoch 3/100
250/250 [==============================] - 4s 16ms/step - loss: 0.3510 - accuracy: 0.8564 - val_loss: 0.3445 - val_accuracy: 0.8625
Epoch 4/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3446 - accuracy: 0.8579 - val_loss: 0.3486 - val_accuracy: 0.8545
Epoch 5/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3400 - accuracy: 0.8649 - val_loss: 0.3473 - val_accuracy: 0.8595
Epoch 6/100
250/250 [==============================] - 4s 15ms/step - loss: 0.3376 - accuracy: 0.8625 - val_loss: 0.3605 - val_accuracy: 0.8570
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3376 - accuracy: 0.8608 - val_loss: 0.3519 - val_accuracy: 0

In [57]:
## load tensorboard extension
%load_ext tensorboard

In [61]:
%tensorboard --logdir logs/fit/20240728-143253

Reusing TensorBoard on port 6007 (pid 15380), started 0:02:10 ago. (Use '!kill 15380' to kill it.)

In [62]:
model.save('model.h5')

c:\Users\NAKUL\Desktop\Data Science masters\Ineuron_ML\Projects_udemy\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
